# Dummy Operation

In [51]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import numpy as np
from scipy import stats
from tqdm import tqdm
import pingouin as pg
from colorama import Fore, Style
from scipy.stats import spearmanr



tqdm.pandas()

In [52]:
# pd.options.mode.dtype_backend = 'pyarrow'
plt.rcParams["figure.figsize"] = (10, 6)
pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
pd.options.display.float_format = '{:.2f}'.format
sns.set_style("whitegrid")

In [53]:
df = pd.read_csv('3_outliers_handled.csv')

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24807 entries, 0 to 24806
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   make_model               24807 non-null  object 
 1   make                     24807 non-null  object 
 2   model                    24807 non-null  object 
 3   price                    24807 non-null  float64
 4   body_type                24807 non-null  object 
 5   age                      24807 non-null  float64
 6   fuel                     24807 non-null  object 
 7   country                  24807 non-null  object 
 8   type                     24807 non-null  object 
 9   warranty                 24807 non-null  object 
 10  mileage                  24807 non-null  float64
 11  gearbox                  24807 non-null  object 
 12  seller                   24807 non-null  object 
 13  engine_size              24807 non-null  float64
 14  gears                 

In [55]:
df.columns

Index(['make_model', 'make', 'model', 'price', 'body_type', 'age', 'fuel',
       'country', 'type', 'warranty', 'mileage', 'gearbox', 'seller',
       'engine_size', 'gears', 'co2_emissions', 'drivetrain', 'cylinders',
       'extras', 'empty_weight', 'full_service_history', 'upholstery',
       'previous_owner', 'energy_efficiency_class', 'power_hp',
       'fuel_consumption_avg', 'address', 'latitude', 'longitude', 'cc', 'ss',
       'em'],
      dtype='object')

In [56]:
categoric_col = df.select_dtypes(include="object")
display(categoric_col.columns)

Index(['make_model', 'make', 'model', 'body_type', 'fuel', 'country', 'type',
       'warranty', 'gearbox', 'seller', 'drivetrain', 'extras',
       'full_service_history', 'upholstery', 'energy_efficiency_class',
       'address', 'cc', 'ss', 'em'],
      dtype='object')

In [57]:
print(df.shape)
df.drop(["address", "latitude", "longitude"], axis=1, inplace=True)
print(df.shape)

(24807, 32)
(24807, 29)


In [58]:
categoric_col = df.select_dtypes(include="object")
display(categoric_col.columns)

Index(['make_model', 'make', 'model', 'body_type', 'fuel', 'country', 'type',
       'warranty', 'gearbox', 'seller', 'drivetrain', 'extras',
       'full_service_history', 'upholstery', 'energy_efficiency_class', 'cc',
       'ss', 'em'],
      dtype='object')

In [59]:
print(Fore.BLUE + Style.BRIGHT + "Number of Uniques:\n" +
      Fore.GREEN + "****"*10 + Style.RESET_ALL + '\n' +
      str(categoric_col.nunique()))

Number of Uniques:
****************************************
make_model                  490
make                         13
model                       488
body_type                     6
fuel                          6
country                      13
type                          4
warranty                      2
gearbox                       3
seller                        2
drivetrain                    3
extras                     9027
full_service_history          2
upholstery                    2
energy_efficiency_class       2
cc                            3
ss                            3
em                            2
dtype: int64


In [60]:
# Encoding all categorical variable (except for "make", "body_type", "fuel_type", seller", drivetrain", "extras", "full_service_history", "upholstery", "energy_efficiency_class", "comfort_convenience_package", "entertainment_media_package", "safety_security_package") in our DataFrame.

df[["make_model_ordinal", "type_ordinal", "gearbox_ordinal",'country_ordinal']] = df[["make_model", "type", "gearbox",'country']].apply(lambda x: pd.factorize(x)[0])
df.head()

,make_model,make,model,price,body_type,age,fuel,country,type,warranty,mileage,gearbox,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,extras,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em,make_model_ordinal,type_ordinal,gearbox_ordinal,country_ordinal
0,Mercedes-Benz A 160,Mercedes-Benz,A 160,16950.00,Compact,6.00,Diesel,Spain,Used,Yes,120200.00,Manual,Dealer,1461.00,5.00,126.00,Front,4.00,Alloy wheels,1295.00,No,Cloth,2.00,efficient,90.00,5.45,premium,Safety Security Premium,Entertainment Media Standard,0,0,0,0
1,Mercedes-Benz EQE 350,Mercedes-Benz,EQE 350,80900.00,Compact,0.00,Electric,Spain,Pre-registered,Yes,5000.00,Automatic,Dealer,1.00,1.00,0.00,4WD,0.00,"Alloy wheels (21""), Shift paddles, Sport packa...",2355.00,No,Part/Full Leather,1.00,efficient,292.00,2.36,premium_plus,Safety Security Premium Plus,Entertainment Media Premium,1,1,1,0
2,Mercedes-Benz A 45 AMG,Mercedes-Benz,A 45 AMG,69900.00,Compact,2.00,Benzine,Spain,Used,Yes,18900.00,Automatic,Dealer,1991.00,8.00,162.00,4WD,4.00,Alloy wheels,1555.00,No,Part/Full Leather,2.00,inefficient,421.00,8.40,premium,Safety Security Standard,Entertainment Media Standard,2,0,1,0
3,Mercedes-Benz A 35 AMG,Mercedes-Benz,A 35 AMG,46990.00,Compact,2.00,Benzine,Spain,Used,Yes,18995.00,Automatic,Dealer,1991.00,7.00,169.00,4WD,4.00,Sport seats,1455.00,No,Part/Full Leather,2.00,inefficient,306.00,7.30,premium,Safety Security Standard,Entertainment Media Standard,3,0,1,0
4,Mercedes-Benz A 45 AMG,Mercedes-Benz,A 45 AMG,16800.00,Compact,7.00,Diesel,Spain,Used,Yes,197000.00,Automatic,Dealer,2143.00,7.00,162.00,4WD,4.00,Alloy wheels,1545.00,No,Part/Full Leather,2.00,inefficient,136.00,4.90,premium,Safety Security Premium,Entertainment Media Standard,2,0,1,0


In [61]:
make_model_ordinal = df.pop("make_model_ordinal")
type_ordinal = df.pop("type_ordinal")
gearbox_ordinal = df.pop("gearbox_ordinal")
country_ordinal = df.pop("country_ordinal")

# Let's insert the columns we have popp
df.insert(1, 'make_model_ordinal', make_model_ordinal)
df.insert(6, "type_ordinal", type_ordinal)
df.insert(11, "gearbox_ordinal", gearbox_ordinal)
df.insert(13, "country_ordinal", country_ordinal)


df.head()

,make_model,make_model_ordinal,make,model,price,body_type,type_ordinal,age,fuel,country,type,gearbox_ordinal,warranty,country_ordinal,mileage,gearbox,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,extras,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em
0,Mercedes-Benz A 160,0,Mercedes-Benz,A 160,16950.00,Compact,0,6.00,Diesel,Spain,Used,0,Yes,0,120200.00,Manual,Dealer,1461.00,5.00,126.00,Front,4.00,Alloy wheels,1295.00,No,Cloth,2.00,efficient,90.00,5.45,premium,Safety Security Premium,Entertainment Media Standard
1,Mercedes-Benz EQE 350,1,Mercedes-Benz,EQE 350,80900.00,Compact,1,0.00,Electric,Spain,Pre-registered,1,Yes,0,5000.00,Automatic,Dealer,1.00,1.00,0.00,4WD,0.00,"Alloy wheels (21""), Shift paddles, Sport packa...",2355.00,No,Part/Full Leather,1.00,efficient,292.00,2.36,premium_plus,Safety Security Premium Plus,Entertainment Media Premium
2,Mercedes-Benz A 45 AMG,2,Mercedes-Benz,A 45 AMG,69900.00,Compact,0,2.00,Benzine,Spain,Used,1,Yes,0,18900.00,Automatic,Dealer,1991.00,8.00,162.00,4WD,4.00,Alloy wheels,1555.00,No,Part/Full Leather,2.00,inefficient,421.00,8.40,premium,Safety Security Standard,Entertainment Media Standard
3,Mercedes-Benz A 35 AMG,3,Mercedes-Benz,A 35 AMG,46990.00,Compact,0,2.00,Benzine,Spain,Used,1,Yes,0,18995.00,Automatic,Dealer,1991.00,7.00,169.00,4WD,4.00,Sport seats,1455.00,No,Part/Full Leather,2.00,inefficient,306.00,7.30,premium,Safety Security Standard,Entertainment Media Standard
4,Mercedes-Benz A 45 AMG,2,Mercedes-Benz,A 45 AMG,16800.00,Compact,0,7.00,Diesel,Spain,Used,1,Yes,0,197000.00,Automatic,Dealer,2143.00,7.00,162.00,4WD,4.00,Alloy wheels,1545.00,No,Part/Full Leather,2.00,inefficient,136.00,4.90,premium,Safety Security Premium,Entertainment Media Standard


In [62]:
print(df.shape)
df.drop(["make_model", "type", 'gearbox','country'], axis=1, inplace=True)
print(df.shape)

df.head()

(24807, 33)
(24807, 29)


,make_model_ordinal,make,model,price,body_type,type_ordinal,age,fuel,gearbox_ordinal,warranty,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,extras,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em
0,0,Mercedes-Benz,A 160,16950.00,Compact,0,6.00,Diesel,0,Yes,0,120200.00,Dealer,1461.00,5.00,126.00,Front,4.00,Alloy wheels,1295.00,No,Cloth,2.00,efficient,90.00,5.45,premium,Safety Security Premium,Entertainment Media Standard
1,1,Mercedes-Benz,EQE 350,80900.00,Compact,1,0.00,Electric,1,Yes,0,5000.00,Dealer,1.00,1.00,0.00,4WD,0.00,"Alloy wheels (21""), Shift paddles, Sport packa...",2355.00,No,Part/Full Leather,1.00,efficient,292.00,2.36,premium_plus,Safety Security Premium Plus,Entertainment Media Premium
2,2,Mercedes-Benz,A 45 AMG,69900.00,Compact,0,2.00,Benzine,1,Yes,0,18900.00,Dealer,1991.00,8.00,162.00,4WD,4.00,Alloy wheels,1555.00,No,Part/Full Leather,2.00,inefficient,421.00,8.40,premium,Safety Security Standard,Entertainment Media Standard
3,3,Mercedes-Benz,A 35 AMG,46990.00,Compact,0,2.00,Benzine,1,Yes,0,18995.00,Dealer,1991.00,7.00,169.00,4WD,4.00,Sport seats,1455.00,No,Part/Full Leather,2.00,inefficient,306.00,7.30,premium,Safety Security Standard,Entertainment Media Standard
4,2,Mercedes-Benz,A 45 AMG,16800.00,Compact,0,7.00,Diesel,1,Yes,0,197000.00,Dealer,2143.00,7.00,162.00,4WD,4.00,Alloy wheels,1545.00,No,Part/Full Leather,2.00,inefficient,136.00,4.90,premium,Safety Security Premium,Entertainment Media Standard


In [63]:
df.dtypes

make_model_ordinal           int64
make                        object
model                       object
price                      float64
body_type                   object
type_ordinal                 int64
age                        float64
fuel                        object
gearbox_ordinal              int64
warranty                    object
country_ordinal              int64
mileage                    float64
seller                      object
engine_size                float64
gears                      float64
co2_emissions              float64
drivetrain                  object
cylinders                  float64
extras                      object
empty_weight               float64
full_service_history        object
upholstery                  object
previous_owner             float64
energy_efficiency_class     object
power_hp                   float64
fuel_consumption_avg       float64
cc                          object
ss                          object
em                  

In [64]:
df.extras.str.count(",").add(1)
#df["extras"].apply(lambda x: x.count(",")).add(1)

0        1
1        6
2        1
3        1
4        1
        ..
24802    1
24803    2
24804    3
24805    2
24806    4
Name: extras, Length: 24807, dtype: int64

In [65]:
df["num_of_extras"] = df["extras"].apply(lambda x: x.count(",")).add(1)

df.head()

,make_model_ordinal,make,model,price,body_type,type_ordinal,age,fuel,gearbox_ordinal,warranty,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,extras,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em,num_of_extras
0,0,Mercedes-Benz,A 160,16950.00,Compact,0,6.00,Diesel,0,Yes,0,120200.00,Dealer,1461.00,5.00,126.00,Front,4.00,Alloy wheels,1295.00,No,Cloth,2.00,efficient,90.00,5.45,premium,Safety Security Premium,Entertainment Media Standard,1
1,1,Mercedes-Benz,EQE 350,80900.00,Compact,1,0.00,Electric,1,Yes,0,5000.00,Dealer,1.00,1.00,0.00,4WD,0.00,"Alloy wheels (21""), Shift paddles, Sport packa...",2355.00,No,Part/Full Leather,1.00,efficient,292.00,2.36,premium_plus,Safety Security Premium Plus,Entertainment Media Premium,6
2,2,Mercedes-Benz,A 45 AMG,69900.00,Compact,0,2.00,Benzine,1,Yes,0,18900.00,Dealer,1991.00,8.00,162.00,4WD,4.00,Alloy wheels,1555.00,No,Part/Full Leather,2.00,inefficient,421.00,8.40,premium,Safety Security Standard,Entertainment Media Standard,1
3,3,Mercedes-Benz,A 35 AMG,46990.00,Compact,0,2.00,Benzine,1,Yes,0,18995.00,Dealer,1991.00,7.00,169.00,4WD,4.00,Sport seats,1455.00,No,Part/Full Leather,2.00,inefficient,306.00,7.30,premium,Safety Security Standard,Entertainment Media Standard,1
4,2,Mercedes-Benz,A 45 AMG,16800.00,Compact,0,7.00,Diesel,1,Yes,0,197000.00,Dealer,2143.00,7.00,162.00,4WD,4.00,Alloy wheels,1545.00,No,Part/Full Leather,2.00,inefficient,136.00,4.90,premium,Safety Security Premium,Entertainment Media Standard,1


In [66]:
print(df.shape)
df.drop("extras", axis=1, inplace=True)
print(df.shape)

(24807, 30)
(24807, 29)


In [67]:
df["fuel"].value_counts()

fuel
Benzine     14913
Diesel       7742
Electric     1793
LPG/CNG       343
Hydrogen       13
Ethanol         3
Name: count, dtype: int64

In [68]:
df.loc[(df.fuel == 'Hydrogen') | (df.fuel == 'Ethanol')].index

Index([ 3112,  5063, 16754, 16767, 16799, 16817, 16819, 16998, 16999, 17019,
       17781, 21895, 21939, 22081, 22128, 22137],
      dtype='int64')

In [69]:
print(df.shape)
df.drop(index=df.loc[(df.fuel == 'Hydrogen') | (df.fuel == 'Ethanol')].index, inplace = True)
print(df.shape)

(24807, 29)
(24791, 29)


In [70]:
df["seller"].value_counts()

seller
Dealer            23323
Private seller     1468
Name: count, dtype: int64

In [71]:
df["drivetrain"].value_counts()

drivetrain
Front    19111
4WD       3581
Rear      2099
Name: count, dtype: int64

In [72]:
df["full_service_history"].value_counts()

full_service_history
No     13673
Yes    11118
Name: count, dtype: int64

In [73]:
df["upholstery"].value_counts()

upholstery
Cloth                13635
Part/Full Leather    11156
Name: count, dtype: int64

In [74]:
df["energy_efficiency_class"].value_counts()

energy_efficiency_class
efficient      15680
inefficient     9111
Name: count, dtype: int64

In [75]:
df["warranty"].value_counts()

warranty
No     12513
Yes    12278
Name: count, dtype: int64

In [76]:
df["fuel"] = df["fuel"].map({"Benzine":0, "Diesel":1, "LPG/CNG":2, "Electric":3})
df["seller"] = df["seller"].map({"Private seller":0, "Dealer":1})
df["drivetrain"] = df["drivetrain"].map({"Rear":0, "Front":1, "4WD":2})
df["full_service_history"] = df["full_service_history"].map({"No":0, "Yes":1})
df["upholstery"] = df["upholstery"].map({"Cloth":0, "Part/Full Leather":1})
df["energy_efficiency_class"] = df["energy_efficiency_class"].map({"inefficient":0, "efficient":1})
df["warranty"] = df["warranty"].map({"No":0, "Yes":1})


In [77]:
df.sample(1)

,make_model_ordinal,make,model,price,body_type,type_ordinal,age,fuel,gearbox_ordinal,warranty,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em,num_of_extras
21159,440,Hyundai,i30,13200.00,Compact,0,4.00,1,0,1,2,96903.00,1,1582.00,6.00,99.00,1,4.00,1238.00,0,0,1.00,1,110.00,3.40,premium,Safety Security Premium,Entertainment Media Premium,2


In [78]:
df["cc"].value_counts()

cc
premium         15834
premium_plus     5945
standard         3012
Name: count, dtype: int64

In [79]:
df["cc"] = df["cc"].astype("category")
df["cc"] = df["cc"].cat.codes


In [80]:
df["cc"].value_counts()

cc
0    15834
1     5945
2     3012
Name: count, dtype: int64

In [81]:
from sklearn.preprocessing import LabelEncoder

In [82]:
# Creating an instance of LabelEncoder() and storing it in labelencoder variable/object
labelencoder = LabelEncoder()

# Applying LabelEncoder for the columns of "safety_security_package" & "entertainment_media_package"
df[["ss_labeled", "em_labeled"]] = df[["ss", "em"]].apply(labelencoder.fit_transform)
df[["ss", "ss_labeled", "em", "em_labeled"]]

,ss,ss_labeled,em,em_labeled
0,Safety Security Premium,0,Entertainment Media Standard,1
1,Safety Security Premium Plus,1,Entertainment Media Premium,0
2,Safety Security Standard,2,Entertainment Media Standard,1
3,Safety Security Standard,2,Entertainment Media Standard,1
4,Safety Security Premium,0,Entertainment Media Standard,1
...,...,...,...,...
24802,Safety Security Standard,2,Entertainment Media Standard,1
24803,Safety Security Premium,0,Entertainment Media Standard,1
24804,Safety Security Premium,0,Entertainment Media Standard,1
24805,Safety Security Premium Plus,1,Entertainment Media Premium,0


In [83]:
display(df[df["ss"] == "Safety Security Premium"][["ss", "ss_labeled"]].sample(5))
display(df[df["ss"] == "Safety Security Premium Plus"][["ss", "ss_labeled"]].sample(5))
display(df[df["ss"] == "Safety Security Standard"][["ss", "ss_labeled"]].sample(5))
display(df[df["em"] == "Entertainment Media Premium"][["em", "em_labeled"]].sample(5))
display(df[df["em"] == "Entertainment Media Standard"][["em", "em_labeled"]].sample(5))

,ss,ss_labeled
6465,Safety Security Premium,0
12608,Safety Security Premium,0
5637,Safety Security Premium,0
1278,Safety Security Premium,0
4059,Safety Security Premium,0


,ss,ss_labeled
24677,Safety Security Premium Plus,1
21968,Safety Security Premium Plus,1
21094,Safety Security Premium Plus,1
8113,Safety Security Premium Plus,1
1416,Safety Security Premium Plus,1


,ss,ss_labeled
4422,Safety Security Standard,2
18981,Safety Security Standard,2
4895,Safety Security Standard,2
22156,Safety Security Standard,2
5278,Safety Security Standard,2


,em,em_labeled
1253,Entertainment Media Premium,0
929,Entertainment Media Premium,0
13306,Entertainment Media Premium,0
2657,Entertainment Media Premium,0
10974,Entertainment Media Premium,0


,em,em_labeled
23028,Entertainment Media Standard,1
7674,Entertainment Media Standard,1
19950,Entertainment Media Standard,1
4565,Entertainment Media Standard,1
19917,Entertainment Media Standard,1


In [84]:
print(df.shape)
df.drop(["ss", "em"], axis=1, inplace=True)
print(df.shape)

(24791, 31)
(24791, 29)


In [85]:
print(df.shape)
df.drop(['model'], axis=1, inplace=True)
print(df.shape)

(24791, 29)
(24791, 28)


In [86]:
for column in ['make', 'body_type']:
    dummies = pd.get_dummies(df[column])
    df[dummies.columns] = dummies
    df.drop(column, axis=1, inplace=True)

In [87]:
df.sample(1)

,make_model_ordinal,price,type_ordinal,age,fuel,gearbox_ordinal,warranty,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,num_of_extras,ss_labeled,em_labeled,Dacia,Fiat,Ford,Hyundai,Mercedes-Benz,Nissan,Opel,Peugeot,Renault,Seat,Skoda,Toyota,Volvo,Compact,Convertible,Coupe,Off-Road/Pick-up,Sedan,Station wagon
6915,246,18499.00,0,3.00,0,0,1,5,29763.00,1,1199.00,6.00,117.00,1,3.00,1090.00,0,0,1.00,0,132.00,4.70,0,4,0,1,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False


In [88]:
df = df.astype('float')

In [89]:
df.sample(1)

,make_model_ordinal,price,type_ordinal,age,fuel,gearbox_ordinal,warranty,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,num_of_extras,ss_labeled,em_labeled,Dacia,Fiat,Ford,Hyundai,Mercedes-Benz,Nissan,Opel,Peugeot,Renault,Seat,Skoda,Toyota,Volvo,Compact,Convertible,Coupe,Off-Road/Pick-up,Sedan,Station wagon
11530,314.00,28990.00,0.00,2.00,1.00,1.00,0.00,1.00,13616.00,1.00,1968.00,7.00,123.00,2.00,4.00,1483.00,1.00,1.00,1.00,1.00,150.00,4.70,0.00,9.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


In [90]:
df.shape

(24791, 45)

## Export dataframe to csv file (dummy)

In [91]:
df.to_csv("dummied.csv", index=False)

In [92]:
df_final = pd.read_csv("dummied.csv")
df_final.head().T

,0,1,2,3,4
make_model_ordinal,0.00,1.00,2.00,3.00,2.00
price,16950.00,80900.00,69900.00,46990.00,16800.00
type_ordinal,0.00,1.00,0.00,0.00,0.00
age,6.00,0.00,2.00,2.00,7.00
fuel,1.00,3.00,0.00,0.00,1.00
gearbox_ordinal,0.00,1.00,1.00,1.00,1.00
warranty,1.00,1.00,1.00,1.00,1.00
country_ordinal,0.00,0.00,0.00,0.00,0.00
mileage,120200.00,5000.00,18900.00,18995.00,197000.00
seller,1.00,1.00,1.00,1.00,1.00


In [93]:
df.columns

Index(['make_model_ordinal', 'price', 'type_ordinal', 'age', 'fuel',
       'gearbox_ordinal', 'warranty', 'country_ordinal', 'mileage', 'seller',
       'engine_size', 'gears', 'co2_emissions', 'drivetrain', 'cylinders',
       'empty_weight', 'full_service_history', 'upholstery', 'previous_owner',
       'energy_efficiency_class', 'power_hp', 'fuel_consumption_avg', 'cc',
       'num_of_extras', 'ss_labeled', 'em_labeled', 'Dacia', 'Fiat', 'Ford',
       'Hyundai', 'Mercedes-Benz', 'Nissan', 'Opel', 'Peugeot', 'Renault',
       'Seat', 'Skoda', 'Toyota', 'Volvo', 'Compact', 'Convertible', 'Coupe',
       'Off-Road/Pick-up', 'Sedan', 'Station wagon'],
      dtype='object')

In [94]:
df1 = df.drop(['Dacia', 'Fiat', 'Ford', 'Hyundai', 'Mercedes-Benz', 'Nissan', 'Opel', 'Peugeot', 'Renault', 'Seat', 'Skoda', 'Toyota', 'Volvo'], axis=1)

df1.to_csv("dummied_make_model.csv", index=False)

pd.read_csv("dummied_make_model.csv")

,make_model_ordinal,price,type_ordinal,age,fuel,gearbox_ordinal,warranty,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,num_of_extras,ss_labeled,em_labeled,Compact,Convertible,Coupe,Off-Road/Pick-up,Sedan,Station wagon
0,0.00,16950.00,0.00,6.00,1.00,0.00,1.00,0.00,120200.00,1.00,1461.00,5.00,126.00,1.00,4.00,1295.00,0.00,0.00,2.00,1.00,90.00,5.45,0.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
1,1.00,80900.00,1.00,0.00,3.00,1.00,1.00,0.00,5000.00,1.00,1.00,1.00,0.00,2.00,0.00,2355.00,0.00,1.00,1.00,1.00,292.00,2.36,1.00,6.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
2,2.00,69900.00,0.00,2.00,0.00,1.00,1.00,0.00,18900.00,1.00,1991.00,8.00,162.00,2.00,4.00,1555.00,0.00,1.00,2.00,0.00,421.00,8.40,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
3,3.00,46990.00,0.00,2.00,0.00,1.00,1.00,0.00,18995.00,1.00,1991.00,7.00,169.00,2.00,4.00,1455.00,0.00,1.00,2.00,0.00,306.00,7.30,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
4,2.00,16800.00,0.00,7.00,1.00,1.00,1.00,0.00,197000.00,1.00,2143.00,7.00,162.00,2.00,4.00,1545.00,0.00,1.00,2.00,0.00,136.00,4.90,0.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24786,480.00,37600.00,0.00,3.00,3.00,1.00,1.00,0.00,40345.00,1.00,1969.00,8.00,0.00,2.00,0.00,2077.00,0.00,1.00,2.00,1.00,392.00,2.00,0.00,1.00,2.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24787,474.00,5499.00,0.00,18.00,0.00,1.00,0.00,4.00,145000.00,1.00,2435.00,5.00,217.00,1.00,5.00,1357.00,1.00,1.00,3.00,1.00,170.00,9.10,0.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24788,476.00,7300.00,0.00,11.00,1.00,0.00,1.00,2.00,96000.00,1.00,1560.00,6.00,99.00,1.00,4.00,1356.00,1.00,0.00,1.00,0.00,114.00,3.80,0.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24789,480.00,29900.00,0.00,5.00,0.00,1.00,1.00,4.00,94343.00,1.00,1969.00,8.00,149.00,1.00,4.00,1617.00,1.00,1.00,1.00,1.00,254.00,6.50,0.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00


In [95]:
df2 = df.drop(['make_model_ordinal'], axis=1)

df2.to_csv("dummied_make.csv", index=False)

pd.read_csv("dummied_make.csv")

,price,type_ordinal,age,fuel,gearbox_ordinal,warranty,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,num_of_extras,ss_labeled,em_labeled,Dacia,Fiat,Ford,Hyundai,Mercedes-Benz,Nissan,Opel,Peugeot,Renault,Seat,Skoda,Toyota,Volvo,Compact,Convertible,Coupe,Off-Road/Pick-up,Sedan,Station wagon
0,16950.00,0.00,6.00,1.00,0.00,1.00,0.00,120200.00,1.00,1461.00,5.00,126.00,1.00,4.00,1295.00,0.00,0.00,2.00,1.00,90.00,5.45,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
1,80900.00,1.00,0.00,3.00,1.00,1.00,0.00,5000.00,1.00,1.00,1.00,0.00,2.00,0.00,2355.00,0.00,1.00,1.00,1.00,292.00,2.36,1.00,6.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
2,69900.00,0.00,2.00,0.00,1.00,1.00,0.00,18900.00,1.00,1991.00,8.00,162.00,2.00,4.00,1555.00,0.00,1.00,2.00,0.00,421.00,8.40,0.00,1.00,2.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
3,46990.00,0.00,2.00,0.00,1.00,1.00,0.00,18995.00,1.00,1991.00,7.00,169.00,2.00,4.00,1455.00,0.00,1.00,2.00,0.00,306.00,7.30,0.00,1.00,2.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
4,16800.00,0.00,7.00,1.00,1.00,1.00,0.00,197000.00,1.00,2143.00,7.00,162.00,2.00,4.00,1545.00,0.00,1.00,2.00,0.00,136.00,4.90,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24786,37600.00,0.00,3.00,3.00,1.00,1.00,0.00,40345.00,1.00,1969.00,8.00,0.00,2.00,0.00,2077.00,0.00,1.00,2.00,1.00,392.00,2.00,0.00,1.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24787,5499.00,0.00,18.00,0.00,1.00,0.00,4.00,145000.00,1.00,2435.00,5.00,217.00,1.00,5.00,1357.00,1.00,1.00,3.00,1.00,170.00,9.10,0.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24788,7300.00,0.00,11.00,1.00,0.00,1.00,2.00,96000.00,1.00,1560.00,6.00,99.00,1.00,4.00,1356.00,1.00,0.00,1.00,0.00,114.00,3.80,0.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24789,29900.00,0.00,5.00,0.00,1.00,1.00,4.00,94343.00,1.00,1969.00,8.00,149.00,1.00,4.00,1617.00,1.00,1.00,1.00,1.00,254.00,6.50,0.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
